# Speech Emotion Recognition - Feature Extraction

Databases used

* The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS)
* Toronto emotional speech set (TESS)

### Import Libraries

Import necessary libraries

In [1]:
import glob
import os
import librosa
import time
import numpy as np
import pandas as pd

### Load all files

We will create our numpy array extracting Mel-frequency cepstral coefficients (MFCCs) while the classes to predict will be extracted from the name of the file.

#### Defining emotions to classify 

Selecting the emotions to be classified. Note that the emotions 'neutral', 'calm' and 'surprised' are only found in RAVDESS dataset and 'pleasantly surprised' or 'ps' is only available in TESS dataset. To combine all the emotions from both datasets into 8 emotion classes, we have changed 'pleasantly surprised' to 'surprised' and 'fearful' to 'fear'

In [2]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fear',
  '07':'disgust',
  '08':'surprised'
}

#defined tess emotions to test on TESS dataset only
tess_emotions=['angry','disgust','fear','ps','happy','sad']

##defined RAVDESS emotions to test on RAVDESS dataset only
ravdess_emotions=['neutral','calm','angry', 'happy','disgust','sad','fear','surprised']

observed_emotions = ['sad','angry','happy','disgust','surprised','neutral','calm','fear']

#### Feature extraction

Using librosa package we can extract the MFCC features. This function loads the file give the file path and after resampling and computing MFCC features, returns the features. We have selected the no. of MFCCs as 40.

https://librosa.org/librosa/generated/librosa.feature.mfcc.html

In [3]:
def extract_feature(file_name, mfcc):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    return result

#### Choosing a dataset

Choose the dataset(s) you want to load using the following function

In [4]:
def dataset_options():
    # choose datasets
    ravdess = True
    tess = True
    ravdess_speech = False
    ravdess_song = False
    data = {'ravdess':ravdess, 'ravdess_speech':ravdess_speech, 'ravdess_song':ravdess_song, 'tess':tess}
    print(data)
    return data

#### Load data

Load data from the datasets required which is obtained by calling the function dataset__options(). Extract features from each file with the selected emotions in chosen datasets using the extract_feature() function defined.

In [5]:
def load_data(test_size=0.2): 
    x,y=[],[]
    
    # feature to extract
    mfcc = True
    
    data = dataset_options()
    paths = []
    if data['ravdess']:
        paths.append("..\Datasets\RAVDESS\*\Actor_*\*.wav")
    elif data['ravdess_speech']:
        paths.append("..\Datasets\RAVDESS\Speech\Actor_*\*.wav")
    elif data['ravdess_song']:
        paths.append("..\Datasets\RAVDESS\Song\Actor_*\*.wav")
        
    for path in paths:
        for file in glob.glob(path):
            file_name=os.path.basename(file)
            emotion=emotions[file_name.split("-")[2]] #to get emotion according to filename. dictionary emotions is defined above.
            if emotion not in observed_emotions: #options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
                continue
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
    if data['tess']:
        for file in glob.glob("..\Datasets\TESS\*AF_*\*.wav"):
            file_name=os.path.basename(file)
            emotion=file_name.split("_")[2][:-4] #split and remove .wav
            if emotion == 'ps':
                emotion = 'surprised'
            if emotion not in observed_emotions: #options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
                continue
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
    return {"X":x,"y":y}

In [6]:
start_time = time.time()

Trial_dict = load_data(test_size = 0.3)

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

{'ravdess': True, 'ravdess_speech': False, 'ravdess_song': False, 'tess': True}
--- Data loaded. Loading time: 0.0024938583374023438 seconds ---


In [7]:
X = pd.DataFrame(Trial_dict["X"])
y = pd.DataFrame(Trial_dict["y"])

In [8]:
X.shape, y.shape

((0, 0), (0, 0))

In [9]:
#renaming the label column to emotion
y=y.rename(columns= {0: 'emotion'})

In [10]:
#concatinating the attributes and label into a single dataframe
data = pd.concat([X, y], axis =1)

In [11]:
data.head()

""


## Shuffling data

In [12]:
#reindexing to shuffle the data at random
data = data.reindex(np.random.permutation(data.index))

In [13]:
# Storing shuffled ravdess and tess data to avoid loading again
data.to_csv("RAVTESS_MFCC_Observed.csv")